In [1]:
import numpy as np
import cvxopt
import struct
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import matplotlib.cm as cmx
import matplotlib.colors as colors

### Control_horizon=1, planning_horizon=1

In [33]:
del_t = 0.1
v_init = 10
w_init = 0.1
a_max = 10
alpha_min = -0.5
alpha_max = 0.5

class agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.xpath = []
        self.ypath = []
        self.vplot = []
        self.wplot = []
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.v)+" "+str(self.w)+" "+str(self.pos))    
            

    def opt_traj(self):
        self.visualize_traj()
        print ("start v: ",self.v,"start w: ",self.w)
        while (np.linalg.norm(self.pos-self.goal)>0.7):
            x_new = self.pos[0] + self.v*del_t*np.cos(self.theta+self.w*del_t)
            y_new = self.pos[1] + self.v*del_t*np.sin(self.theta+self.w*del_t)
            self.theta += self.w*del_t
            self.xpath.append(x_new)
            self.ypath.append(y_new)
            self.pos = [x_new,y_new]
            theta_new = self.theta
            v_new = self.v
            w_new = self.w
            v_orig = self.v
            w_orig = self.w
            diff_v = 999
            diff_w = 999
#             xgi = self.pos[0]
#             ygi = self.pos[1]
            tol = 1e-1
            while (np.linalg.norm(np.array([diff_w,diff_v])**2)>tol):
                # vi = self.v, wi = self.w
#                 theta_new += self.w*del_t
#                 xgi += (self.v*del_t*np.cos(theta_new) - self.goal[0])
#                 ygi += (self.v*del_t*np.sin(theta_new) - self.goal[1])
                w_guess = self.w
                v_guess = self.v
                x = del_t*np.cos(self.theta+w_guess*del_t)
                a = -v_guess*del_t**2*np.sin(self.theta+w_guess*del_t)
                y = del_t*np.sin(self.theta+w_guess*del_t)
                b = v_guess*del_t**2*np.cos(self.theta+w_guess*del_t)
                kx = self.pos[0]+v_guess*del_t**2*w_guess*np.sin(self.theta+w_guess*del_t) - self.goal[0]
                ky = self.pos[1]-v_guess*del_t**2*w_guess*np.cos(self.theta+w_guess*del_t) - self.goal[1]
                arrx = np.array([[x],[a]])
                arry = np.array([[y],[b]])
                p_mat = 2*(np.matmul(arrx,arrx.T)+np.matmul(arry,arry.T))
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = 2*(kx*arrx+ky*arry)
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.array([[0,-1],[-1,0],[1,0],[0,1],[1,0],[-1,0],[0,1],[0,-1]])
                h = cvxopt.matrix(np.array([self.wmax,0,self.vmax,self.wmax,v_guess+5,-(v_guess-5),w_guess+0.1,-(w_guess-0.1)]),tc='d')
#                 print (arrx,arry,kx,ky)
                g = cvxopt.matrix(g_mat,tc='d')
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][0] #vd
                w_new = sol['x'][1] #wd
#                 print ("Optimizer op:",v_new,w_new)
#                 print (v_new,w_new)
                diff_v = v_new - self.v
                diff_w = w_new - self.w
                print (diff_v,diff_w)
                self.v = v_new #vd+vi
                self.w = w_new #wd+wi
            print ("#####################End of Optimization######################")
#             print ("new v: ",self.v,"new w: ",self.w)
#             print ("Opti complete")
#             self.vplot.append(self.v)
#             self.wplot.append(self.w)
            print (self)
            self.time+=1
            self.visualize_traj()

             
    def visualize_traj(self):
        figure = plt.figure()
#         print (self.pos)
        print ("Theta: ",self.theta)
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Rectangle(
            (self.pos[0]-self.rad*np.sqrt(2)*np.cos(self.theta+np.pi/4),self.pos[1]-self.rad*np.sqrt(2)*np.sin(self.theta+np.pi/4)),
            height = self.rad*2,
            width = self.rad*2,
            angle = self.theta*180/np.pi,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.plot(self.xpath,self.ypath,'b-')
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 100.0)
        ax.set_ylim(-10.0, 100.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


In [37]:
bot = agent(2,np.array([0,0]),np.array([80,50]),np.pi/2,20,0.5)
bot.opt_traj()

Theta:  1.5707963267948966
start v:  10 start w:  0.1
5.000000003192696 -0.0999999122265636
4.999999627382866 -0.09999988231174561
3.3631284424018304e-08 -0.09999982092591811
#####################End of Optimization######################
0 1.5807963267948966 19.999999664206847 -0.19999961546422731 [-0.009999833334166612, 0.9999500004166653]
Theta:  1.5807963267948966
5.040142525558622e-08 -0.09999984772272777
#####################End of Optimization######################
1 1.5607963652484738 19.999999714608272 -0.2999994631869551 [0.009999756095071479, 2.9998499684414166]
Theta:  1.5607963652484738
6.19865474504877e-08 -0.09999994964590159
#####################End of Optimization######################
2 1.5307964189297782 19.99999977659482 -0.39999941283285667 [0.08997823920471247, 4.998250160615845]
Theta:  1.5307964189297782
-1.343707360490498e-06 -0.0999999925232471
#####################End of Optimization######################
3 1.4907964776464926 19.99999843288746 -0.4999994053561

-1.1324348747621116e-07 0.09999976710783864
#####################End of Optimization######################
36 0.12076620352567705 19.99999985062943 0.2999503104306263 [47.34284425027735, 43.80217132166577]
Theta:  0.12076620352567705
-5.56467377066383e-07 0.0999881305749592
#####################End of Optimization######################
37 0.1507612345687397 19.999999294162052 0.3999384410055855 [49.320158303488, 44.102552871078245]
Theta:  0.1507612345687397
5.474297246621518e-07 -0.09993471129898795
#####################End of Optimization######################
38 0.19075507866929825 19.999999841591777 0.30000372970659756 [51.28388093791634, 44.481753520733086]
Theta:  0.19075507866929825
-2.0661394017906787e-07 -0.0999901855308131
#####################End of Optimization######################
39 0.22075545163995802 19.999999634977836 0.20001354417578446 [53.235345540347566, 44.91968712539781]
Theta:  0.22075545163995802
-5.213818248250845e-07 -0.09999418241871642
####################

### Control_horizon = 1, planning_horizon = p

In [76]:
del_t = 0.1
planning_horizon = 10
v_guess = 20*np.ones((planning_horizon))
w_guess = 0.1*np.ones((planning_horizon))
v_init = v_guess[0]
w_init = w_guess[0]
a_max = 10
alpha_min = -0.5
alpha_max = 0.5

class mpc_agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.xpath = []
        self.ypath = []
        self.vplot = []
        self.wplot = []
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.v_guess = v_guess
        self.w_guess = w_guess
        self.prev_v = v_guess
        self.prev_w = w_guess
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        plt.ion()
        self.dummy_x = np.array([])
        self.dummy_y = np.array([])
        
        
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.v)+" "+str(self.w)+" "+str(self.pos))    
            
    def p_constructor(self):
#         print (self.w_guess)
        wg = np.cumsum(self.w_guess)
        p_theta = self.theta+wg*del_t
        
        xs = del_t*np.cos(p_theta)
        qx = np.sin(p_theta)
        bs = -self.v_guess*del_t**2*qx
        brev = bs[::-1]
        dterms = np.cumsum(brev)
        dterms = dterms[::-1]
        arrx = np.concatenate((xs, dterms), axis=None)
        ksx = -bs*self.w_guess
        kx = np.sum(ksx) + self.pos[0] - self.goal[0]
        
        ys = del_t*np.sin(p_theta)
        qy = np.cos(p_theta)
        by = self.v_guess*del_t**2*qy
        byrev = by[::-1]
        dyterms = np.cumsum(byrev)
        dyterms = dyterms[::-1]
        arry = np.concatenate((ys, dyterms), axis=None)
        ksy = -by*self.w_guess
        ky = np.sum(ksy) + self.pos[1] - self.goal[1]
        
        
        return arrx,arry,kx,ky
        
    def opt_traj(self):
        self.visualize_traj()
        print ("start v: ",self.v,"start w: ",self.w)
        
        while (np.linalg.norm(self.pos-self.goal)>0.7):
            x_new = self.pos[0] + self.v*del_t*np.cos(self.theta+self.w*del_t)
            y_new = self.pos[1] + self.v*del_t*np.sin(self.theta+self.w*del_t)
            self.theta += self.w*del_t
            self.xpath.append(x_new)
            self.ypath.append(y_new)
            self.pos = [x_new,y_new]
            theta_new = self.theta
            v_new = self.v
            w_new = self.w
            diff_v = 999
            diff_w = 999
            tol = 1e-1
            
            while (np.linalg.norm(np.array([diff_w,diff_v])**2)>tol):
                arrx,arry,kx,ky = self.p_constructor()
                arrx = arrx.reshape((-1,1))
                arry = arry.reshape((-1,1))
#                 print (arrx.shape,arry.shape,kx.shape,ky.shape)
                p_mat = 2*(np.matmul(arrx,arrx.T)+np.matmul(arry,arry.T))
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = 2*(kx*arrx+ky*arry)
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.eye(2*planning_horizon)
                g_mat = np.concatenate((g_mat,-np.eye(2*planning_horizon)),axis=0)
                g_mat = np.concatenate((g_mat,np.eye(2*planning_horizon)),axis=0)
                g_mat = np.concatenate((g_mat,-np.eye(2*planning_horizon)),axis=0)
                h_mat = np.concatenate((self.vmax*np.ones(planning_horizon),self.wmax*np.ones(planning_horizon),np.zeros(planning_horizon),self.wmax*np.ones(planning_horizon),self.v_guess+5,self.w_guess+0.1,-(self.v_guess-5),-(self.w_guess-0.1)),axis=None)
                h = cvxopt.matrix(h_mat,tc='d')
#                 print ("h_mat:",h_mat)
#                 print ("p_mat:",p_mat)
#                 print (arrx,arry,kx,ky)
                g = cvxopt.matrix(g_mat,tc='d')
#                 print (p_mat.shape,q_mat.shape,g_mat.shape,h_mat.shape)
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][:planning_horizon] #vd
                w_new = sol['x'][planning_horizon:] #wd
#                 print ("Optimizer op:",v_new,w_new)
                diff_v = v_new[0] - self.v
                diff_w = w_new[0] - self.w
#                 print (diff_v,diff_w)
                self.v = v_new[0]
                self.w = w_new[0]
                self.prev_v = np.array(v_new)
                self.prev_w = np.array(w_new)
#                 print (self.v,self.w)
                self.v_guess = self.prev_v[1:]
                self.v_guess = np.concatenate((self.v_guess,np.array([[self.v]])),axis=0)
                self.v_guess = self.v_guess.reshape((-1,))
                self.w_guess = self.prev_w[1:]
                self.w_guess = np.concatenate((self.w_guess,np.array([[self.w]])),axis=0)
                self.w_guess = self.w_guess.reshape((-1,))
#                 self.v_guess = 10*np.ones((planning_horizon))
#                 self.w_guess = 0.1*np.ones((planning_horizon))
#                 print (self.v_guess,self.w_guess)
                
            print ("##########End of Optimization#############")
            print ("new v: ",self.v,"new w: ",self.w)
            print ("Opti complete")
            print (self)
            self.time+=1
            self.visualize_traj()

             
    def visualize_traj(self):
        figure = plt.figure()
#         print (self.pos)
        print ("Theta: ",self.theta)
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Rectangle(
            (self.pos[0]-self.rad*np.sqrt(2)*np.cos(self.theta+np.pi/4),self.pos[1]-self.rad*np.sqrt(2)*np.sin(self.theta+np.pi/4)),
            height = self.rad*2,
            width = self.rad*2,
            angle = self.theta*180/np.pi,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.plot(self.xpath,self.ypath,'b-')
#         ax.plot(self.prev_v,self.ypath,'b-')
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 100.0)
        ax.set_ylim(-10.0, 100.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


In [78]:
bot1 = mpc_agent(2,np.array([0,0]),np.array([60,90]),np.pi/4,30,0.5)
bot1.opt_traj()

Theta:  0.7853981633974483
start v:  20.0 start w:  0.1
##########End of Optimization#############
new v:  29.99999994982747 new w:  0.39999864017786496
Opti complete
0 0.7953981633974483 29.99999994982747 0.39999864017786496 [1.400000952361581, 1.4282847522068791]
Theta:  0.7953981633974483
##########End of Optimization#############
new v:  29.99999925492052 new w:  0.4999947378726051
Opti complete
1 0.8353980274152348 29.99999925492052 0.4999947378726051 [3.412648668497223, 3.652975548841429]
Theta:  0.8353980274152348
##########End of Optimization#############
new v:  29.999999946493105 new w:  0.3999953597596147
Opti complete
2 0.8853975012024954 29.999999946493105 0.3999953597596147 [5.3115940793141005, 5.97547546911911]
Theta:  0.8853975012024954
##########End of Optimization#############
new v:  29.999999173812864 new w:  0.499968885136218
Opti complete
3 0.9253970371784569 29.999999173812864 0.499968885136218 [7.116146464554994, 8.372054417707549]
Theta:  0.9253970371784569
###

##########End of Optimization#############
new v:  10.948371996897206 new w:  -0.0009905989734834213
Opti complete
35 1.012554610939872 10.948371996897206 -0.0009905989734834213 [54.193440316853774, 80.71828013920374]
Theta:  1.012554610939872
##########End of Optimization#############
new v:  9.853509814738917 new w:  -0.0011169806973223958
Opti complete
36 1.0124555510425237 9.853509814738917 -0.0011169806973223958 [54.77346281942475, 81.64685021751541]
Theta:  1.0124555510425237
##########End of Optimization#############
new v:  8.868158632049067 new w:  -0.0010038004422876956
Opti complete
37 1.0123438529727915 8.868158632049067 -0.0010038004422876956 [55.29557509226169, 82.48250285542461]
Theta:  1.0123438529727915
##########End of Optimization#############
new v:  7.981348204432672 new w:  -0.0008301174982519197
Opti complete
38 1.0122434729285628 7.981348204432672 -0.0008301174982519197 [55.76555161934766, 83.23454304000259]
Theta:  1.0122434729285628
##########End of Optimizati

### Planning horizon = p, control horizon = c with obstacle

In [632]:
del_t = 0.1
planning_horizon = 40
control_horizon = 5
v_guess = np.ones((planning_horizon))
# w_guess = 0.2*np.ones((planning_horizon))
# v_guess = 1+np.random.randint(20, size=(planning_horizon))
w_guess = 0.1*(np.random.rand(planning_horizon)-0.5)
print (v_guess,w_guess)
# w_guess = np.random.randint(0.1, size=(planning_horizon))
v_init = v_guess
w_init = w_guess
a_max = 10
obst_rad = 2
obst_pos = [30,0]
alpha_min = -0.5
alpha_max = 0.5

class multi_mpc_agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.obst_rad = obst_rad
        self.obst = obst_pos
        self.xpath = []
        self.ypath = []
        self.vplot = []
        self.wplot = []
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.v_guess = v_guess
        self.w_guess = w_guess
        self.prev_v = v_guess
        self.prev_w = w_guess
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        plt.ion()
        self.dummy_x = np.array([])
        self.dummy_y = np.array([])
        
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.pos))    
            
    def p_constructor(self):
#         print (self.w_guess)
        wg = np.cumsum(self.w_guess)
        p_theta = self.theta+wg*del_t #net theta
        
        #overall row structure is [x y z a+b+c b+c c]
        xs = del_t*np.cos(p_theta) #v coefficients (i.e, x,y,z)
        qx = np.sin(p_theta)
        bs = -self.v_guess*del_t**2*qx #w coefficients (i.e. a,b,c)
        brev = bs[::-1] #this gives [c,b,a]
        dterms = np.cumsum(brev) #this gives [c,b+c,a+b+c]
        dterms = dterms[::-1] #this gives [a+b+c,b+c,c]
        arrx = np.concatenate((xs, dterms), axis=None)
        ksx = -dterms*self.w_guess 
        kx = np.sum(ksx) + self.pos[0] - self.goal[0] 
        
        
        ys = del_t*np.sin(p_theta)
        qy = np.cos(p_theta)
        by = self.v_guess*del_t**2*qy
        byrev = by[::-1]
        dyterms = np.cumsum(byrev)
        dyterms = dyterms[::-1]
        arry = np.concatenate((ys, dyterms), axis=None)
        ksy = -dyterms*self.w_guess 
        ky = np.sum(ksy) + self.pos[1] - self.goal[1] 
        
        
        
        return arrx,arry,kx,ky
    
    def obst_constraint(self,i):
        
        wg = np.cumsum(self.w_guess[:i])
        p_theta = self.theta+wg*del_t
        
        xs = del_t*np.cos(p_theta)
        qx = np.sin(p_theta)
        bs = -self.v_guess[:i]*del_t**2*qx
        xk = 2*((self.pos[0] - self.obst[0]) + np.sum(self.v_guess[:i]*del_t*np.cos(p_theta)))
        brev = bs[::-1]
        dterms = np.cumsum(brev)
        dterms = dterms[::-1]
        arrx = np.concatenate((xs, dterms), axis=None)
        ksx = dterms*self.w_guess[:i]
        kx = -np.sum(ksx) - np.sum(self.v_guess[:i]*del_t*np.cos(p_theta))
        nx = np.sum(xk*kx)
        consx = (self.pos[0]+np.sum(self.v_guess[:i]*del_t*np.cos(p_theta))-self.obst[0])**2
        
        ys = del_t*np.sin(p_theta)
        qy = np.cos(p_theta)
        by = self.v_guess[:i]*del_t**2*qy
        yk = 2*((self.pos[1] - self.obst[1]) + np.sum(self.v_guess[:i]*del_t*np.sin(p_theta)))
        byrev = by[::-1]
        dyterms = np.cumsum(byrev)
        dyterms = dyterms[::-1]
        arry = np.concatenate((ys, dyterms), axis=None)
        ksy = dyterms*self.w_guess[:i]
        ky = -np.sum(ksy) - np.sum(self.v_guess[:i]*del_t*np.sin(p_theta))
        ny = np.sum(yk*ky)
        consy = (self.pos[1]+np.sum(self.v_guess[:i]*del_t*np.sin(p_theta))-self.obst[1])**2
        
        px = xk*arrx
        py = yk*arry
#         print ("px: ",px.shape,"py: ",py.shape)
        
        obsx = np.concatenate(((px[:i]+py[:i]),np.zeros(planning_horizon-i),(px[i:]+py[i:]),np.zeros(planning_horizon-i)),axis=None)
        
        return (obsx,nx,ny,consx,consy)
    
    def is_pos_def(self,x):
        return np.all(np.linalg.eigvals(x) >= 0)

    def opt_traj(self):
        self.visualize_traj()
        print ("start v: ",self.v,"start w: ",self.w)
        
        while (np.linalg.norm(self.pos-self.goal)>0.6):
            v_new = self.v
            w_new = self.w
            diff_v = 999
            diff_w = 999
            tol = 1e-1
            
            while (np.linalg.norm(np.array([diff_w,diff_v])**2)>tol):
                arrx,arry,kx,ky = self.p_constructor()
                arrx = arrx.reshape((-1,1))
                arry = arry.reshape((-1,1))
#                 print (arrx.shape,arry.shape,kx.shape,ky.shape)
                p_mat = 2*(np.matmul(arrx,arrx.T)+np.matmul(arry,arry.T))
                
                p_mat += 1e-6*np.eye(p_mat.shape[0])
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = 2*(kx*arrx+ky*arry) 
#                 if (self.is_pos_def(p_mat)):
#                     print ("Yes psd")
#                 else:
#                     print ("Not psd")
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.eye(2*planning_horizon)
                g_mat = np.concatenate((g_mat,-np.eye(2*planning_horizon)),axis=0)
                g_mat = np.concatenate((g_mat,np.eye(2*planning_horizon)),axis=0)
                g_mat = np.concatenate((g_mat,-np.eye(2*planning_horizon)),axis=0)
                h_mat = np.concatenate((self.vmax*np.ones(planning_horizon),self.wmax*np.ones(planning_horizon),np.zeros(planning_horizon),self.wmax*np.ones(planning_horizon),self.v_guess+2,self.w_guess+0.05,-(self.v_guess-2),-(self.w_guess-0.05)),axis=None)
                ##For collision avoidance
                if ((self.pos[0]-self.obst[0])<5 or (self.pos[1]-self.obst[1])<5):
                    for i in range(planning_horizon):
                        obst_xk,obst_nx,obst_ny,consx,consy = self.obst_constraint(i+1)
    #                     print (obst_xk)
    #                     print (obst_xk.shape,obst_nx.shape)
                        g_mat = np.concatenate((g_mat,-np.array([obst_xk])),axis=0)
                        h_mat = np.concatenate((h_mat,-(-consx-consy+(self.obst_rad+self.rad+2)**2+0.5-obst_nx-obst_ny)),axis=None)
    #                 print (g_mat)
    #                 print (h_mat)
                h = cvxopt.matrix(h_mat,tc='d')
                g = cvxopt.matrix(g_mat,tc='d')
                init_vals = np.concatenate((self.v_guess,self.w_guess),axis=None)
#                 print (init_vals)
#                 print (g_mat)
#                 print (p_mat.shape,q_mat.shape,g_mat.shape,h_mat.shape)
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][:planning_horizon] #vd
                w_new = sol['x'][planning_horizon:] #wd
#                 print ("Optimizer op:",v_new,w_new)
#                 print ("Opti going on...")
                diff_v = np.linalg.norm(np.array(v_new) - self.v)
                diff_w = np.linalg.norm(np.array(w_new) - self.w)
#                 print (diff_v,diff_w)
                self.v = v_new
                self.w = w_new
                self.prev_v = np.array(v_new)
                self.prev_w = np.array(w_new)
#                 print (self.v,self.w)
                
                self.v_guess = self.prev_v
                self.v_guess = self.v_guess.reshape((-1,))
                self.w_guess = self.prev_w
                self.w_guess = self.w_guess.reshape((-1,))
            
                
            print ("##########End of Optimization#############")
#             print ("new v: ",self.v,"new w: ",self.w)
            print ("Opti complete")
        
            dummy_x = []
            dummy_y = []
            temp_x = self.pos[0]
            temp_y = self.pos[1]
            temp_theta = self.theta
            for i in range(planning_horizon):
                temp_x = temp_x + self.v[i]*del_t*np.cos(temp_theta+self.w[i]*del_t)
                temp_y = temp_y + self.v[i]*del_t*np.sin(temp_theta+self.w[i]*del_t)
                temp_theta += self.w[i]*del_t
                dummy_x.append(temp_x)
                dummy_y.append(temp_y)

            self.dummy_x = np.array(dummy_x)
            self.dummy_y = np.array(dummy_y)
            
            for i in range(control_horizon):
                x_new = self.pos[0] + self.v[i]*del_t*np.cos(self.theta+self.w[i]*del_t)
                y_new = self.pos[1] + self.v[i]*del_t*np.sin(self.theta+self.w[i]*del_t)
                self.theta += self.w[i]*del_t
                self.xpath.append(x_new)
                self.ypath.append(y_new)
                self.pos = [x_new,y_new]
                print (self)
                self.time+=1
                self.visualize_traj()
            print ("#####End of control horizon#####")
            self.v_guess = self.v[control_horizon:planning_horizon]
            self.v_guess = np.concatenate((self.v_guess,self.v[planning_horizon-1]*np.ones([control_horizon,1])),axis=0)
            self.v_guess = self.v_guess.reshape((-1,))
            self.w_guess = self.w[control_horizon:planning_horizon]
            self.w_guess = np.concatenate((self.w_guess,self.w[planning_horizon-1]*np.ones([control_horizon,1])),axis=0)
            self.w_guess = self.w_guess.reshape((-1,))
                

             
    def visualize_traj(self):
        figure = plt.figure()
#         print (self.pos)
        print ("Theta: ",self.theta)
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Rectangle(
            (self.pos[0]-self.rad*np.sqrt(2)*np.cos(self.theta+np.pi/4),self.pos[1]-self.rad*np.sqrt(2)*np.sin(self.theta+np.pi/4)),
            height = self.rad*2,
            width = self.rad*2,
            angle = self.theta*180/np.pi,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        obst = matplotlib.patches.Rectangle(
            (self.obst[0]-self.obst_rad,self.obst[1]-self.obst_rad),
            height = self.obst_rad*2,
            width = self.obst_rad*2,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        ax.add_patch(obst)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="yellow", markersize =15,linewidth=3.0)
        ax.plot(self.xpath,self.ypath,'g-')
        ax.plot(self.dummy_x, self.dummy_y, 'r--')
#         ax.plot(self.prev_v,self.ypath,'b-')
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 120.0)
        ax.set_ylim(-10.0, 120.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] [-0.01000348 -0.03306196 -0.01036294 -0.00761984  0.03624689 -0.04466732
  0.01902458 -0.03082739 -0.02108491  0.04701711 -0.03182606  0.01821234
  0.00636284 -0.0297582  -0.04140217  0.02466123  0.04387743  0.04405953
 -0.0149704  -0.00961297  0.03970269  0.04484043  0.01618113 -0.03963111
 -0.0476212   0.00057772  0.0178234   0.03703725  0.02027825  0.01153969
  0.02449639 -0.03719512  0.00652262  0.03645185 -0.03898622  0.02378964
  0.02360265 -0.03641453  0.03956878 -0.04553991]


In [633]:
# [ 1.50000000e+01  1.50000000e+01  5.00000000e-01  5.00000000e-01
#   0.00000000e+00  0.00000000e+00  5.00000000e-01  5.00000000e-01
#   1.50000000e+01  1.50000000e+01  1.50000000e-01  1.50000000e-01
#  -5.00000000e+00 -5.00000000e+00 -5.00000000e-02 -5.00000000e-02
#   1.26400143e+02  2.61765134e+02]

In [634]:
bot2 = multi_mpc_agent(2,np.array([0,0]),np.array([100,0]),0,10,0.7)
bot2.opt_traj()

Theta:  0
start v:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] start w:  [-0.01000348 -0.03306196 -0.01036294 -0.00761984  0.03624689 -0.04466732
  0.01902458 -0.03082739 -0.02108491  0.04701711 -0.03182606  0.01821234
  0.00636284 -0.0297582  -0.04140217  0.02466123  0.04387743  0.04405953
 -0.0149704  -0.00961297  0.03970269  0.04484043  0.01618113 -0.03963111
 -0.0476212   0.00057772  0.0178234   0.03703725  0.02027825  0.01153969
  0.02449639 -0.03719512  0.00652262  0.03645185 -0.03898622  0.02378964
  0.02360265 -0.03641453  0.03956878 -0.04553991]
##########End of Optimization#############
Opti complete
0 -0.03874702919449389 [0.9992493822580985, -0.03873733279679933]
Theta:  -0.03874702919449389
1 -0.07965388673628498 [1.9960786437365865, -0.11830701212998274]
Theta:  -0.07965388673628498
2 -0.11837471547025119 [2.9890804906819106, -0.2364054602731948]
Theta:  -0.11837471547025119
3 -0.1568087035883

69 0.07673418663054495 [68.72937259696985, -2.923235953587161]
Theta:  0.07673418663054495
#####End of control horizon#####
##########End of Optimization#############
Opti complete
70 0.07775049162522511 [69.50873044305251, -2.862518099277314]
Theta:  0.07775049162522511
71 0.07376558568534158 [70.34749144529805, -2.800533935809151]
Theta:  0.07376558568534158
72 0.07733875534066241 [71.06669839238528, -2.7448002016074895]
Theta:  0.07733875534066241
73 0.08111426948530158 [71.8549526224758, -2.680720936846498]
Theta:  0.08111426948530158
74 0.08568110710031848 [72.58786668946841, -2.617769926318483]
Theta:  0.08568110710031848
#####End of control horizon#####
##########End of Optimization#############
Opti complete
75 0.11691743897023932 [73.27439848152517, -2.5371346323913953]
Theta:  0.11691743897023932
76 0.16420926977733272 [73.97494180846178, -2.4210536817410717]
Theta:  0.16420926977733272
77 0.19810192322940567 [74.71092457904103, -2.273316385014949]
Theta:  0.19810192322940567

143 0.01175309393653639 [95.63373111733088, -0.0828604892839799]
Theta:  0.01175309393653639
144 0.012200883095103306 [95.75378155008153, -0.08139569530391805]
Theta:  0.012200883095103306
#####End of control horizon#####
##########End of Optimization#############
Opti complete
145 0.013026989201895791 [95.8533521245323, -0.08009851712678726]
Theta:  0.013026989201895791
146 0.013845070450402244 [95.95778043701915, -0.07865260739533675]
Theta:  0.013845070450402244
147 0.014649834997183848 [96.06271525288183, -0.07711521967239365]
Theta:  0.014649834997183848
148 0.015433903992877663 [96.1765800345705, -0.07535770201136316]
Theta:  0.015433903992877663
149 0.016179118437855536 [96.28004400043774, -0.07368360017787101]
Theta:  0.016179118437855536
#####End of control horizon#####
##########End of Optimization#############
Opti complete
150 0.01744907987094211 [96.39303436994321, -0.071711822075646]
Theta:  0.01744907987094211
151 0.018661647635332807 [96.4849590106713, -0.06999615765264

217 0.015783150808828198 [99.41685520473821, -0.009277395882156774]
Theta:  0.015783150808828198
218 0.01579358062975803 [99.4331737803683, -0.009019645710997682]
Theta:  0.01579358062975803
219 0.015803662105485252 [99.44883067126139, -0.008772188896105005]
Theta:  0.015803662105485252
#####End of control horizon#####


### To check

In [161]:
import sympy as sym

In [187]:
x0 = sym.Symbol('x0')
y0 = sym.Symbol('y0')
theta0 = sym.Symbol('theta0')
t = sym.Symbol('t')
v1 = sym.Symbol('v1')
w1 = sym.Symbol('w1')
v2 = sym.Symbol('v2')
w2 = sym.Symbol('w2')
v3 = sym.Symbol('v3')
w3 = sym.Symbol('w3')
x1 = sym.Symbol('x1')
y1 = sym.Symbol('y1')
x2 = sym.Symbol('x2')
y2 = sym.Symbol('y2')
x3 = sym.Symbol('x3')
y3 = sym.Symbol('y3')
v_1g = sym.Symbol('v_1g')
w_1g = sym.Symbol('w_1g')
v_2g = sym.Symbol('v_2g')
w_2g = sym.Symbol('w_2g')
v_3g = sym.Symbol('v_3g')
w_3g = sym.Symbol('w_3g')

In [191]:
a = sym.expand(t*sym.cos(theta0+w_1g*t))
b = sym.expand(t*sym.cos(theta0+(w_1g+w_2g)*t))
c = sym.expand(t*sym.cos(theta0+(w_1g+w_2g+w_3g)*t))
x = sym.expand(-v_1g*t*t*sym.sin(theta0+w_1g*t))
y = sym.expand(-v_2g*t*t*sym.sin(theta0+(w_1g+w_2g)*t))
z = sym.expand(-v_3g*t*t*sym.sin(theta0+(w_1g+w_2g+w_3g)*t))

In [203]:
s1 = sym.simplify(a*v1+b*v2+c*v3+(y+x+z)*w1+(y+z)*w2+z*w3+x0-(y+x+z)*w_1g-(y+z)*w_2g-z*w_3g)

In [201]:
x3 = x0 + v1*t*sym.cos(theta0+w1*t)+ v2*t*sym.cos(theta0+(w1+w2)*t) + v3*t*sym.cos(theta0+(w1+w2+w3)*t)
x_3g = x0 + v_1g*t*sym.cos(theta0+w_1g*t)+ v_2g*t*sym.cos(theta0+(w_1g+w_2g)*t) + v_3g*t*sym.cos(theta0+(w_1g+w_2g+w_3g)*t)
coeff_v1 = sym.diff(x_3g,v_1g)
coeff_w1 = sym.diff(x_3g,w_1g)
coeff_v2 = sym.diff(x_3g,v_2g)
coeff_w2 = sym.diff(x_3g,w_2g)
coeff_v3 = sym.diff(x_3g,v_3g)
coeff_w3 = sym.diff(x_3g,w_3g)

In [204]:
s2 = sym.simplify(x_3g+coeff_v1*(v1-v_1g)+coeff_w1*(w1-w_1g)+coeff_v2*(v2-v_2g)+coeff_w2*(w2-w_2g)+coeff_v3*(v3-v_3g)+coeff_w3*(w3-w_3g))

In [206]:
s2

-t**2*v_3g*(w3 - w_3g)*sin(t*(w_1g + w_2g + w_3g) + theta0) - t**2*(w1 - w_1g)*(v_1g*sin(t*w_1g + theta0) + v_2g*sin(t*(w_1g + w_2g) + theta0) + v_3g*sin(t*(w_1g + w_2g + w_3g) + theta0)) - t**2*(w2 - w_2g)*(v_2g*sin(t*(w_1g + w_2g) + theta0) + v_3g*sin(t*(w_1g + w_2g + w_3g) + theta0)) + t*v_1g*cos(t*w_1g + theta0) + t*v_2g*cos(t*(w_1g + w_2g) + theta0) + t*v_3g*cos(t*(w_1g + w_2g + w_3g) + theta0) + t*(v1 - v_1g)*cos(t*w_1g + theta0) + t*(v2 - v_2g)*cos(t*(w_1g + w_2g) + theta0) + t*(v3 - v_3g)*cos(t*(w_1g + w_2g + w_3g) + theta0) + x0

In [212]:
s1

-t**2*v_3g*w3*sin(t*w_1g + t*w_2g + t*w_3g + theta0) + t**2*v_3g*w_3g*sin(t*w_1g + t*w_2g + t*w_3g + theta0) - t**2*w1*(v_1g*sin(t*w_1g + theta0) + v_2g*sin(t*w_1g + t*w_2g + theta0) + v_3g*sin(t*w_1g + t*w_2g + t*w_3g + theta0)) - t**2*w2*(v_2g*sin(t*w_1g + t*w_2g + theta0) + v_3g*sin(t*w_1g + t*w_2g + t*w_3g + theta0)) + t**2*w_1g*(v_1g*sin(t*w_1g + theta0) + v_2g*sin(t*w_1g + t*w_2g + theta0) + v_3g*sin(t*w_1g + t*w_2g + t*w_3g + theta0)) + t**2*w_2g*(v_2g*sin(t*w_1g + t*w_2g + theta0) + v_3g*sin(t*w_1g + t*w_2g + t*w_3g + theta0)) + t*v1*cos(t*w_1g + theta0) + t*v2*cos(t*w_1g + t*w_2g + theta0) + t*v3*cos(t*w_1g + t*w_2g + t*w_3g + theta0) + x0